# 순환 신경망
 - 지금까지 다룬 딥러닝 모델로는 다룰 수 없는 문제들이 있습니다. 지난 데이터들을 사용하여 미래의 데이터를 예측하는 문제입니다.
 - 우리는 이러한 문제를 해결하기 위해 신경망에 시간의 개념을 도입합니다.
 - 이러한 데이터를 시계열 데이터라 부르고 시계열 데이터를 다루는 데 사용하는 모델을 순환 신경망이라 합니다.

# 사인파
 - 사인파를 생각해봅시다.
 - f(T) = sin(2 * pi * t / T) (t = 1, ..., 2T), T는 주기
 - 그래프를 생각해보면 값이 반복적으로 무한히 연속된다는 사실을 알 수 있습니다.
 - 여기에 노이즈를 추가해보겠습니다.
 - f(T) = sin(2 * pi * t / T) + 0.05 * u, u는 -1부터1까지의 균등분포를 가지는 난수
 - 그래프를 보면 마치 잡음이 섞인 소리의 파형처럼 표현된다는 것을 알 수 있습니다.
 - 우리는 이러한 데이터들을 바탕으로 미래의 데이터를 예측하는 모델을 만들어 보려 합니다.

# 과거의 은닉층, 오차함수, 활성화 함수
 - 지난 데이터들이 미래의 데이터를 결정하는 데 영향을 미쳐야 하므로 각 데이터들을 모델안에 저장해야합니다. 이를 과거의 은닉층이라 정의합니다.
 - 이제까지는 오차함수를 엔트로피 오차함수와 출력층에서는 활성화 함수로 소프트맥스 함수나 시그모이드 함수를 사용했지만 출력이 확률이 아닌 함수값이어야 하므로 다른 식을 사용해야 합니다.
 - 간단하게 생각해서 활성화 함수는 y = Vh(t) + c처럼 선형활성인 식을 생각해보고 오차함수도 모델의 예측값과 정답인 값의 차의 제곱을 모두 더한 후 반을 나눈 식을 사용해보도록 합시다. 이를 제곱오차함수(squared error function)이라 합니다.
 - 이 경우도 지난 모델들과 같이 역전파법으로 오차를 계산합니다. 이를 Backpropagation Through Time(BPTT)라 합니다.

# 학습 데이터 셋 준비
 - 노이즈가 포함된 사인파를 사용하겠습니다.

In [1]:
import numpy as np

def sin(x, T=100):
    return np.sin(2.0 * np.pi * x / T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2 * T + 1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

T = 100
f = toy_problem(T) # t = 0, ... , 200일 때의 데이터 생성

In [2]:
length_of_sequences = 2 * T # 시계열 전체의 길이
maxlen = 25 # 하나의 시계열 데이터의 길이

data = []
target = []
for i in range(0, length_of_sequences - maxlen + 1): # 과거의 데이터 셋을 t - maxlen + 1 라 생각
    data.append(f[i:i + maxlen])
    target.append(f[i + maxlen])

In [3]:
# 데이터 차원 맞추기

X = np.array(data).reshape(len(data), maxlen, 1) # 데이터 수, maxlen, 입력차원(1)
Y = np.array(target).reshape(len(data), 1) # 출력차원은 1

In [4]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 학습데이터 준비

N_train = int(len(data) * 0.9)
N_validation = len(data) - N_train

X_train, X_validaion, Y_train, Y_validaion = train_test_split(X, Y, test_size=N_validation)

# 텐서플로로 구현

In [5]:
def inference(x, n_batch, maxlen=None, n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)
        
    cell = tf.contrib.rnn.BasicRNNCell(n_hidden) # cell에 은닉층 값을 저장하여 시간순으로 데이터를 넘겨주어 순전파하도록 합니다.
    initial_state = cell.zero_state(n_batch, tf.float32)
    
    state = initial_state
    outputs = [] # 과거의 은닉층의 출력을 저장
    with tf.variable_scope('RNN'): # 저장된 outputs 에는 RNN이 붙음, 이를 재이용한다는 것을 명시
        for t in range(maxlen):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
            (cell_output, state) = cell(x[:, t, :], state)
            outputs.append(cell_output)
            
    output = outputs[-1]
    
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    y = tf.matmul(output, V) + c
    
    return y

In [6]:
def loss(y, t):
    mse = tf.reduce_mean(tf.square(y - t))
    return mse

In [7]:
def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999) # 학습법은 Adam을 사용합니다.
    train_step = optimizer.minimize(loss)
    return train_step

In [8]:
tf.reset_default_graph()

n_in = len(X[0][0]) # 1
n_hidden = 20
n_out = len(Y[0]) # 1

x = tf.placeholder(tf.float32, shape=[None, maxlen, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
n_batch = tf.placeholder(tf.int32, [], name='batch_size')

y = inference(x, n_batch, maxlen=maxlen, n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# 학습

epochs = 500
batch_size = 5

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = N_train # 배치 사이즈

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            n_batch: batch_size
        })
        
    # 검증 데이터를 사용해 평가
    val_loss = loss.eval(session=sess, feed_dict={
        x: X_validation,
        t: Y_validation,
        n_bath: N_validation
    })
    
    history['val_loss'].appned(val_loss)
    print('epoch:', epoch, ' validation loss:', val_loss)
    
    # 얼리 스탑핑 검사
    if early_stopping.validate(val_loss):
        break

InvalidArgumentError: _MklConcatOp : Dimensions of inputs should match: shape[0][0]= 3 vs. shape[1][0] = 5
	 [[node RNN/basic_rnn_cell/concat (defined at <ipython-input-5-edfe3745bbf8>:19) ]]

Errors may have originated from an input operation.
Input Source operations connected to node RNN/basic_rnn_cell/concat:
 BasicRNNCellZeroState/zeros (defined at <ipython-input-5-edfe3745bbf8>:11)

Original stack trace for 'RNN/basic_rnn_cell/concat':
  File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "D:\Anaconda\lib\site-packages\t